In [1]:
!pip install requests

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\abdul\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [23]:
import requests
import json

# Base URL for Elasticsearch running on port 8989
BASE_URL = "http://localhost:8989"

# Function to create a collection (index) in Elasticsearch
def createCollection(p_collection_name):
    response = requests.put(f"{BASE_URL}/{p_collection_name}")
    if response.status_code == 200:
        print(f"Collection '{p_collection_name}' created successfully.")
    else:
        print(f"Failed to create collection '{p_collection_name}': {response.text}")

# Function to index data into the specified collection, excluding a given column
def indexData(p_collection_name, p_exclude_column, dataset):
    for record in dataset:
        # Remove the excluded column from the record
        if p_exclude_column in record:
            del record[p_exclude_column]
        
        # Index the record
        response = requests.post(f"{BASE_URL}/{p_collection_name}/_doc", json=record)
        if response.status_code == 201:
            print(f"Document indexed in collection '{p_collection_name}'.")
        else:
            print(f"Failed to index document: {response.text}")

# Function to search records by a column value in the specified collection
def searchByColumn(p_collection_name, p_column_name, p_column_value):
    query = {
        "query": {
            "term": {
                p_column_name: p_column_value
            }
        }
    }
    response = requests.post(f"{BASE_URL}/{p_collection_name}/_search", json=query)
    if response.status_code == 200:
        results = response.json()
        return results['hits']['hits']
    else:
        print(f"Failed to search: {response.text}")
        return None

# Function to get the count of employees in the specified collection
def getEmpCount(p_collection_name):
    response = requests.get(f"{BASE_URL}/{p_collection_name}/_count")
    if response.status_code == 200:
        count = response.json().get('count')
        return count
    else:
        print(f"Failed to get count: {response.text}")
        return None

# Function to delete an employee by ID in the specified collection
def delEmpById(p_collection_name, p_employee_id):
    response = requests.delete(f"{BASE_URL}/{p_collection_name}/_doc/{p_employee_id}")
    if response.status_code == 200:
        print(f"Employee with ID '{p_employee_id}' deleted successfully.")
    else:
        print(f"Failed to delete employee: {response.text}")

# Function to get the department facet (count of employees by department)
def getDepFacet(p_collection_name):
    query = {
        "size": 0,
        "aggs": {
            "department_count": {
                "terms": {
                    "field": "Department.keyword"
                }
            }
        }
    }
    response = requests.post(f"{BASE_URL}/{p_collection_name}/_search", json=query)
    if response.status_code == 200:
        return response.json()['aggregations']['department_count']['buckets']
    else:
        print(f"Failed to get department facet: {response.text}")
        return None


In [24]:
v_nameCollection = "hash_abdulakram"
v_phoneCollection = "hash_2967"


In [25]:
createCollection(v_nameCollection)
createCollection(v_phoneCollection)


Collection 'abdulakram' created successfully.
Failed to create collection 'E02967': {"error":{"root_cause":[{"type":"invalid_index_name_exception","reason":"Invalid index name [E02967], must be lowercase","index_uuid":"_na_","index":"E02967"}],"type":"invalid_index_name_exception","reason":"Invalid index name [E02967], must be lowercase","index_uuid":"_na_","index":"E02967"},"status":400}


In [17]:
import pandas as pd

# Load dataset from CSV and convert to dictionary format
dataset = pd.read_csv("employee_data.csv")
dataset.dropna(inplace=True)
dataset = dataset.to_dict(orient='records')

In [18]:
indexData(v_nameCollection, 'Department', dataset)
indexData(v_phoneCollection, 'Gender', dataset)


Document indexed in collection 'hash_abdulakram'.
Document indexed in collection 'hash_abdulakram'.
Document indexed in collection 'hash_abdulakram'.
Document indexed in collection 'hash_abdulakram'.
Document indexed in collection 'hash_abdulakram'.
Document indexed in collection 'hash_abdulakram'.
Document indexed in collection 'hash_abdulakram'.
Document indexed in collection 'hash_abdulakram'.
Document indexed in collection 'hash_abdulakram'.
Document indexed in collection 'hash_abdulakram'.
Document indexed in collection 'hash_abdulakram'.
Document indexed in collection 'hash_abdulakram'.
Document indexed in collection 'hash_abdulakram'.
Document indexed in collection 'hash_abdulakram'.
Document indexed in collection 'hash_abdulakram'.
Document indexed in collection 'hash_abdulakram'.
Document indexed in collection 'hash_abdulakram'.
Document indexed in collection 'hash_abdulakram'.
Document indexed in collection 'hash_abdulakram'.
Document indexed in collection 'hash_abdulakram'.


In [19]:
print("Employee count:", getEmpCount(v_nameCollection))


Employee count: 102


In [20]:
delEmpById(v_nameCollection, "E02003")


Failed to delete employee: {"_index":"hash_abdulakram","_id":"E02003","_version":1,"result":"not_found","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":102,"_primary_term":1}


In [21]:
print("Search by Department 'IT':", searchByColumn(v_nameCollection, "Department", "IT"))
print("Search by Gender 'Male':", searchByColumn(v_nameCollection, "Gender", "Male"))
print("Search by Department 'IT' in phone collection:", searchByColumn(v_phoneCollection, "Department", "IT"))


Search by Department 'IT': []
Search by Gender 'Male': []
Search by Department 'IT' in phone collection: []


In [22]:
print("Department Facet:", getDepFacet(v_nameCollection))
print("Department Facet in phone collection:", getDepFacet(v_phoneCollection))


Department Facet: []
Department Facet in phone collection: []
